Общий план задания
1. Скачать [http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/](http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/) файлы соответствующие хромосомам 1-22.
2. Перевести с помощью plink https://www.cog-genomics.org/plink/1.9/ из vcf в bed\bim\fam
3. Оставить в данных только SNP с помощью plink.
4. Слить полученные файлы в один набор bed\bim\fam
5. Изменить fam файл, чтобы в нем была информация о популяциях. Соответствие образец-популяция можно найти на сайт 1000 геномов.
6. Сделать новый набор данных, чтобы в нем были только европейские популяции.
7. http://software.genetics.ucla.edu/admixture/ - с помощью этой программы сделать анализ по 5 компонентам (базовый запуск можно найти в документации).

Как результат ожидается следующее:

а. Текстовый файл, какие команды использовались для каждого шага (чтобы можно было воспроизвести).

б.Q файл по итогам 7 шага

в. Графики разложения отельных индивидов на компоненты

#### 1. Скачать http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ файлы соответствующие хромосомам 1-22.
в google colaboratory скачал:

!echo http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr{1..22}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi | xargs -P22 -n1 wget -nv

In [ ]:
for i in range(1, 23):
    !wget http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr{str(i)}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz

#### 2. Перевести с помощью plink https://www.cog-genomics.org/plink/1.9/ из vcf в bed\bim\fam
#### 3. Оставить в данных только SNP с помощью plink.

In [17]:
# Устанавливаем PLINK
!wget http://s3.amazonaws.com/plink2-assets/alpha2/plink2_linux_x86_64.zip
!unzip plink_linux_x86_64_20200121.zip -d plink/

--2020-02-06 00:17:52--  http://s3.amazonaws.com/plink2-assets/alpha2/plink2_linux_x86_64.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.137.222
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.137.222|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8879151 (8.5M) [application/zip]
Saving to: ‘plink2_linux_x86_64.zip’

plink2_linux_x86_64 100%[===================>]   8.47M  7.59MB/s    in 1.1s    

2020-02-06 00:17:59 (7.59 MB/s) - ‘plink2_linux_x86_64.zip’ saved [8879151/8879151]

Archive:  plink_linux_x86_64_20200121.zip
  inflating: plink/plink             
  inflating: plink/LICENSE           
  inflating: plink/toy.ped           
  inflating: plink/toy.map           
  inflating: plink/prettify          


In [18]:
#clear_output()
for chr_num in range(1, 23):
    #Распаковываем хромосому в bed-bim-fam файлы
    !plink/plink --vcf ALL.chr{str(chr_num)}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz --make-bed --out ALL.chr{str(chr_num)}
    #Оставляем только SNP
    !plink/plink --bfile ALL.chr{str(chr_num)} --snps-only --make-bed --out ALL.chr{str(chr_num)}
    #Удаляем дубликаты одинаковых SNPs
    !cut -f2 ALL.chr{str(chr_num)}.bim | sort | uniq -d > dup{str(chr_num)}.snps
    !echo найдено дупликатов в chr{str(chr_num)} хромосоме: 
    !wc -l dup{str(chr_num)}.snps
    !echo ' - удаляю...'
    !plink/plink --bfile ALL.chr{str(chr_num)} --exclude dup{str(chr_num)}.snps --make-bed --out ALL.chr{str(chr_num)}                        

PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr1.log.
Options in effect:
  --make-bed
  --out ALL.chr1
  --vcf ALL.chr1.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz

32167 MB RAM detected; reserving 16083 MB for main workspace.
--vcf: ALL.chr1-temporary.bed + ALL.chr1-temporary.bim + ALL.chr1-temporary.fam
written.
6468094 variants loaded from .bim file.
2504 people (0 males, 0 females, 2504 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ALL.chr1.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2504 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999841.


--vcf: ALL.chr3-temporary.bed + ALL.chr3-temporary.bim + ALL.chr3-temporary.fam
written.
5832276 variants loaded from .bim file.
2504 people (0 males, 0 females, 2504 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ALL.chr3.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2504 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999832.
5832276 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr3.bed + ALL.chr3.bim + ALL.chr3.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999838.
5265763 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr5.bed + ALL.chr5.bim + ALL.chr5.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr5.log.
Options in effect:
  --bfile ALL.chr5
  --make-bed
  --out ALL.chr5
  --snps-only

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
5055536 out of 5265763 variants loaded from .bim file.
2504 people (0 males,

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999834.
4716715 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr7.bed + ALL.chr7.bim + ALL.chr7.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr7.log.
Options in effect:
  --bfile ALL.chr7
  --make-bed
  --out ALL.chr7
  --snps-only

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
4533180 out of 4716715 variants loaded from .bim file.
2504 people (0 males,

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999838.
3560687 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr9.bed + ALL.chr9.bim + ALL.chr9.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr9.log.
Options in effect:
  --bfile ALL.chr9
  --make-bed
  --out ALL.chr9
  --snps-only

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
3427241 out of 3560687 variants loaded from .bim file.
2504 people (0 males,

...

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999814.
2857916 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr13.bed + ALL.chr13.bim + ALL.chr13.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr13.log.
Options in effect:
  --bfile ALL.chr13
  --make-bed
  --out ALL.chr13
  --snps-only

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
2737034 out of 2857916 variants loaded from .bim file.
2504 people (0 

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999833.
2424689 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr15.bed + ALL.chr15.bim + ALL.chr15.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr15.log.
Options in effect:
  --bfile ALL.chr15
  --make-bed
  --out ALL.chr15
  --snps-only

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
2328557 out of 2424689 variants loaded from .bim file.
2504 people (0 

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999842.
2329288 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr17.bed + ALL.chr17.bim + ALL.chr17.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr17.log.
Options in effect:
  --bfile ALL.chr17
  --make-bed
  --out ALL.chr17
  --snps-only

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
2234710 out of 2329288 variants loaded from .bim file.
2504 people (0 

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999827.
1832506 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr19.bed + ALL.chr19.bim + ALL.chr19.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr19.log.
Options in effect:
  --bfile ALL.chr19
  --make-bed
  --out ALL.chr19
  --snps-only

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
1758443 out of 1832506 variants loaded from .bim file.
2504 people (0 

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999787.
1105538 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr21.bed + ALL.chr21.bim + ALL.chr21.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr21.log.
Options in effect:
  --bfile ALL.chr21
  --make-bed
  --out ALL.chr21
  --snps-only

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
1058549 out of 1105538 variants loaded from .bim file.
2504 people (0 

#### 4. Слить полученные файлы в один набор bed\bim\fam

In [46]:
#Создаем список всех файлов для объединения в один набор
#!rm merge_list.txt
for i in range(2, 23):
    with open('merge_list.txt', 'a') as mlist:
        mlist.write(f'ALL.chr{str(i)}.bed ALL.chr{str(i)}.bim ALL.chr{str(i)}.fam \n')

In [47]:
!cat merge_list.txt

ALL.chr2.bed ALL.chr2.bim ALL.chr2.fam 
ALL.chr3.bed ALL.chr3.bim ALL.chr3.fam 
ALL.chr4.bed ALL.chr4.bim ALL.chr4.fam 
ALL.chr5.bed ALL.chr5.bim ALL.chr5.fam 
ALL.chr6.bed ALL.chr6.bim ALL.chr6.fam 
ALL.chr7.bed ALL.chr7.bim ALL.chr7.fam 
ALL.chr8.bed ALL.chr8.bim ALL.chr8.fam 
ALL.chr9.bed ALL.chr9.bim ALL.chr9.fam 
ALL.chr10.bed ALL.chr10.bim ALL.chr10.fam 
ALL.chr11.bed ALL.chr11.bim ALL.chr11.fam 
ALL.chr12.bed ALL.chr12.bim ALL.chr12.fam 
ALL.chr13.bed ALL.chr13.bim ALL.chr13.fam 
ALL.chr14.bed ALL.chr14.bim ALL.chr14.fam 
ALL.chr15.bed ALL.chr15.bim ALL.chr15.fam 
ALL.chr16.bed ALL.chr16.bim ALL.chr16.fam 
ALL.chr17.bed ALL.chr17.bim ALL.chr17.fam 
ALL.chr18.bed ALL.chr18.bim ALL.chr18.fam 
ALL.chr19.bed ALL.chr19.bim ALL.chr19.fam 
ALL.chr20.bed ALL.chr20.bim ALL.chr20.fam 
ALL.chr21.bed ALL.chr21.bim ALL.chr21.fam 
ALL.chr22.bed ALL.chr22.bim ALL.chr22.fam 


In [19]:
#Пробуем объединить все файлы в один набор
!plink/plink --threads 6 --bfile ALL.chr1 --merge-list merge_list.txt --make-bed --out ALL.merged

PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.merged.log.
Options in effect:
  --bfile ALL.chr1
  --make-bed
  --merge-list merge_list.txt
  --out ALL.merged
  --threads 6

32167 MB RAM detected; reserving 16083 MB for main workspace.
to length-80+ variant IDs; consider using a different naming scheme for long
indels and the like.
Error: 3 variants with 3+ alleles present.
* If you believe this is due to strand inconsistency, try --flip with
  ALL.merged-merge.missnp.
  (Warning: if this seems to work, strand errors involving SNPs with A/T or C/G
  alleles probably remain in your data.  If LD between nearby SNPs is high,
  --flip-scan should detect them.)
* If you are dealing with genuine multiallelic variants, we recommend exporting
  that subset of the data to VCF (via e.g. '--recode vcf'), merging with
  another tool/script, and then importing the result; PL

Найдены трёхаллельные варианты
Нужно их удалить из файлов
немного подредактируем файл ALL.merged-merge.missnp

In [21]:
#Удаляем трёхаллельные варианты
for chr_num in range(1, 23):
    !plink/plink --bfile ALL.chr{str(chr_num)} --exclude ALL.merged-merge.missnp --make-bed --out ALL.chr{str(chr_num)}    

PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr1.log.
Options in effect:
  --bfile ALL.chr1
  --exclude ALL.merged-merge.missnp
  --make-bed
  --out ALL.chr1

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
6215983 variants loaded from .bim file.
2504 people (0 males, 0 females, 2504 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ALL.chr1.nosex .
--exclude: 6215981 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2504 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999953.
6215981 variants and 2504 people pass filters and QC.
Note: No phe

4533180 variants loaded from .bim file.
2504 people (0 males, 0 females, 2504 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ALL.chr7.nosex .
--exclude: 4533180 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2504 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.99995.
4533180 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr7.bed + ALL.chr7.bim + ALL.chr7.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun

--exclude: 2737032 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2504 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999951.
2737032 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr13.bed + ALL.chr13.bim + ALL.chr13.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr14.log.
Options in effect:
  --bfile ALL.chr14
  --exclude ALL.merged-m

Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999942.
1758439 variants and 2504 people pass filters and QC.
Note: No phenotypes present.
--make-bed to ALL.chr19.bed + ALL.chr19.bim + ALL.chr19.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.chr20.log.
Options in effect:
  --bfile ALL.chr20
  --exclude ALL.merged-merge.missnp
  --make-bed
  --out ALL.chr20

32167 MB RAM detected; reserving 16083 MB for main workspace.
filenames.
1745169 variants loaded from .bim file.
2504 peo

In [27]:
#Заново сливаем все файлы в один набор
!plink/plink --threads 6 --memory 24576 --bfile ALL.chr1 --merge-list merge_list.txt --make-bed --out ALL.merged

PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.merged.log.
Options in effect:
  --bfile ALL.chr1
  --make-bed
  --memory 24576
  --merge-list merge_list.txt
  --out ALL.merged
  --threads 6

32167 MB RAM detected; reserving 24576 MB for main workspace.
Performing single-pass merge (2504 people, 18627896 variants).
Merged fileset written to ALL.merged-merge.bed + ALL.merged-merge.bim +
ALL.merged-merge.fam .
18627896 variants loaded from .bim file.
2504 people (0 males, 0 females, 2504 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ALL.merged.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2504 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828

#### 5. Изменить fam файл, чтобы в нем была информация о популяциях. Соответствие образец-популяция можно найти на сайт 1000 геномов.

In [ ]:
#Скачиваем таблицу, где можно найти соответствие образец-популяция.
!wget 'http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/working/20130606_sample_info/20130606_sample_info.xlsx'

In [28]:
#Устанавливаем и импортируем необходимые библиотеки для работы с таблицей
#!pip install pandas
#!pip install xlrd
import pandas as pd

In [29]:
#Читаем лист "Sample info"
df = pd.read_excel('20130606_sample_info.xlsx', sheet_name = 'Sample Info', header = 0, usecols = ['Sample', 'Population', 'Population Description'])

In [30]:
#Таблица соответствий
for pop, desc in zip(df['Population'].unique(), df['Population Description'].unique()):
    print (f'{pop} - {desc}')

GBR - British in England and Scotland
FIN - Finnish in Finland
CHS - Southern Han Chinese, China
PUR - Puerto Rican in Puerto Rico
CDX - Chinese Dai in Xishuangbanna, China
CLM - Colombian in Medellin, Colombia
IBS - Iberian populations in Spain
PEL - Peruvian in Lima, Peru
PJL - Punjabi in Lahore,Pakistan
KHV - Kinh in Ho Chi Minh City, Vietnam
ACB - African Caribbean in Barbados
GWD - Gambian in Western Division, The Gambia
ESN - Esan in Nigeria
BEB - Bengali in Bangladesh
MSL - Mende in Sierra Leone
STU - Sri Lankan Tamil in the UK
ITU - Indian Telugu in the UK
CEU - Utah residents with Northern and Western European ancestry
YRI - Yoruba in Ibadan, Nigeria
CHB - Han Chinese in Bejing, China
JPT - Japanese in Tokyo, Japan
LWK - Luhya in Webuye, Kenya
ASW - African Ancestry in Southwest US
MXL - Mexican Ancestry in Los Angeles, California
TSI - Toscani in Italy
GIH - Gujarati Indian in Houston,TX


In [31]:
#Создаем словарь с числовой кодировкой популяций
#Европейские кодируем числом больше > 100
pop_cod = {}
for cnt, samp in enumerate(df.Population.unique()):
    if samp in ['GBR', 'FIN', 'IBS', 'TSI']:
        pop_cod[samp] = cnt + 3 + 100
    else:
        pop_cod[samp] = cnt + 3

In [32]:
#Кодирую популяцию каждого образца в число по ID
df['Popnum'] = 0
ind_cod = {}
for cnt in range(len(df)):
    df.iloc[cnt, 3] = pop_cod[df.Population[cnt]]
    ind_cod[df.iloc[cnt,0]] = pop_cod[df.Population[cnt]]

In [33]:
#Считываем fam - файл
df_fam = pd.read_csv('ALL.merged.fam', sep = ' ', header = None, index_col = False)

In [34]:
#Меняем обозначение популяции на число из словаря
for cnt in range(len(df_fam)):
    df_fam.iloc[cnt, 5] = ind_cod[df_fam.iloc[cnt,0]]

In [35]:
#Перезаписываем fam - файл
df_fam.to_csv('ALL.merged.fam', sep = ' ', header = False, index = False, )

#### 6. Сделать новый набор данных, чтобы в нем были только европейские популяции.

Создаём список европейских популяций

In [36]:
df_keep = df_fam.loc[df_fam[5] > 100][1]

In [37]:
print ('Количество европейских образцов = ', len(df_keep))

Количество европейских образцов =  404


In [38]:
df_keep.to_csv('keep.fam', sep = ' ', header = False, index = False, )

Делаем набор из европейских популяций

In [39]:
!plink/plink --bfile ALL.merged --keep-fam keep.fam --make-bed --out ALL.merged.eu

PLINK v1.90b6.15 64-bit (21 Jan 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ALL.merged.eu.log.
Options in effect:
  --bfile ALL.merged
  --keep-fam keep.fam
  --make-bed
  --out ALL.merged.eu

32167 MB RAM detected; reserving 16083 MB for main workspace.
18627896 variants loaded from .bim file.
2504 people (0 males, 0 females, 2504 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ALL.merged.eu.nosex .
2504 phenotype values loaded from .fam.
--keep-fam: 404 people remaining.
phenotypes to be ignored, use the --allow-no-sex flag.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 404 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping ra

#### 7. http://software.genetics.ucla.edu/admixture/ - с помощью этой программы сделать анализ по 5 компонентам (базовый запуск можно найти в документации).

In [ ]:
!wget http://software.genetics.ucla.edu/admixture/binaries/admixture_linux-1.3.0.tar.gz

In [ ]:
!tar -xvf admixture_linux-1.3.0.tar.gz

Запускаем на 4 итерации.

In [40]:
!adm/admixture -C 4 ALL.merged.eu.bed 5 -j6

****                   ADMIXTURE Version 1.3.0                  ****
****                    Copyright 2008-2015                     ****
****           David Alexander, Suyash Shringarpure,            ****
****                John  Novembre, Ken Lange                   ****
****                                                            ****
****                 Please cite our paper!                     ****
****   Information at www.genetics.ucla.edu/software/admixture  ****

Parallel execution requested.  Will use 6 threads.
Random seed: 43
Point estimation method: Block relaxation algorithm
Convergence acceleration algorithm: QuasiNewton, 3 secant conditions
Point estimation will terminate at 4 iterations
Estimation of standard errors disabled; will compute point estimates only.
Size of G: 404x18627896
Performing five EM steps to prime main algorithm
1 (EM) 	Elapsed: 284.002	Loglikelihood: -7.00867e+08	(delta): 1.07426e+10
2 (EM) 	Elapsed: 291.45	Loglikelihood: -6.68232e+08	(delta

##### Q файл по итогам 7 шага

In [41]:
!head ALL.merged.eu.5.Q

0.171859 0.799915 0.016705 0.000010 0.011512
0.449836 0.321868 0.000547 0.024397 0.203353
0.002067 0.834965 0.162063 0.000010 0.000894
0.000010 0.229830 0.352719 0.163078 0.254363
0.000980 0.327470 0.087150 0.036509 0.547892
0.034665 0.693854 0.000010 0.000010 0.271461
0.000010 0.985035 0.014007 0.000010 0.000938
0.628153 0.351371 0.003244 0.017223 0.000010
0.153980 0.824602 0.015864 0.005544 0.000010
0.000765 0.193234 0.005483 0.001074 0.799443


###### Графики разложения отельных индивидов на компоненты

In [42]:
df_Q = pd.read_csv('ALL.merged.eu.5.Q', names = ['comp1', 'comp2', 'comp3', 'comp4', 'comp5'], sep = ' ', header = None, index_col = False)
df_efam = df_fam = pd.read_csv('ALL.merged.europ.fam', sep = ' ', header = None, index_col = False)
df_Q.set_index(df_efam[0])

,comp1,comp2,comp3,comp4,comp5
0,,,,,
HG00096,0.171859,0.799915,0.016705,0.000010,0.011512
HG00097,0.449836,0.321868,0.000547,0.024397,0.203353
HG00099,0.002067,0.834965,0.162063,0.000010,0.000894
HG00100,0.000010,0.229830,0.352719,0.163078,0.254363
HG00101,0.000980,0.327470,0.087150,0.036509,0.547892
...,...,...,...,...,...
NA20822,0.000010,0.000010,0.000010,0.332391,0.667579
NA20826,0.002142,0.000957,0.190126,0.806765,0.000010
NA20827,0.437640,0.000595,0.000010,0.559146,0.002609


In [ ]:
!pip install plotly.express

In [46]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [43]:
cod_pop = dict(zip(pop_cod.values(), pop_cod.keys()))

In [44]:
sub_names = []
for cnt in range(len(df_efam)):
    sub_names.append(f'{df_efam.iloc[cnt,0]} - {cod_pop[df_efam.iloc[cnt, 5]]}')

In [56]:
start = 0
end = 40
cols = 4
fig = make_subplots(rows = 10, 
                    cols = cols,
                    subplot_titles = sub_names[start:end],                    
                    #vertical_spacing = 0.08,
                    #horizontal_spacing = None
                   )
for i in range(40):
    fig.append_trace(go.Bar(
        #x = ['comp1', 'comp2', 'comp3', 'comp4', 'comp5'],
        y = df_Q.iloc[i, :],
        hovertext = df_Q.iloc[i, :]
    ), row = i // cols + 1, col = i % cols + 1)
    #print(i // 4 + 1, i % 4 + 1, df_Q.iloc[i, :])
#fig.update_layout(height = 300 + len(df_Q)//4 + 1, width=1000, title_text="Stacked subplots")
fig.update_layout(height = 100 * 10, 
                  width=200 * cols, 
                  title_text="Графики разложения отельных индивидов на компоненты",
                  showlegend = False
                 )
fig.show()

In [57]:
start = 40
end = 80
cols = 4
fig = make_subplots(rows = 10, 
                    cols = cols,
                    subplot_titles = sub_names[start:end],                    
                    #vertical_spacing = 0.08,
                    #horizontal_spacing = None
                   )
for i in range(40):
    fig.append_trace(go.Bar(
        #x = ['comp1', 'comp2', 'comp3', 'comp4', 'comp5'],
        y = df_Q.iloc[i, :],
        hovertext = df_Q.iloc[i, :]
    ), row = i // cols + 1, col = i % cols + 1)
    #print(i // 4 + 1, i % 4 + 1, df_Q.iloc[i, :])
#fig.update_layout(height = 300 + len(df_Q)//4 + 1, width=1000, title_text="Stacked subplots")
fig.update_layout(height = 100 * 10, 
                  width=200 * cols, 
                  title_text="Графики разложения отельных индивидов на компоненты",
                  showlegend = False
                 )
fig.show()

In [58]:
start = 80
end = 120
cols = 4
fig = make_subplots(rows = 10, 
                    cols = cols,
                    subplot_titles = sub_names[start:end],                    
                    #vertical_spacing = 0.08,
                    #horizontal_spacing = None
                   )
for i in range(40):
    fig.append_trace(go.Bar(
        #x = ['comp1', 'comp2', 'comp3', 'comp4', 'comp5'],
        y = df_Q.iloc[i, :],
        hovertext = df_Q.iloc[i, :]
    ), row = i // cols + 1, col = i % cols + 1)
    #print(i // 4 + 1, i % 4 + 1, df_Q.iloc[i, :])
#fig.update_layout(height = 300 + len(df_Q)//4 + 1, width=1000, title_text="Stacked subplots")
fig.update_layout(height = 100 * 10, 
                  width=200 * cols, 
                  title_text="Графики разложения отельных индивидов на компоненты",
                  showlegend = False
                 )
fig.show()

In [59]:
start = 120
end = 160
cols = 4
fig = make_subplots(rows = 10, 
                    cols = cols,
                    subplot_titles = sub_names[start:end],                    
                    #vertical_spacing = 0.08,
                    #horizontal_spacing = None
                   )
for i in range(40):
    fig.append_trace(go.Bar(
        #x = ['comp1', 'comp2', 'comp3', 'comp4', 'comp5'],
        y = df_Q.iloc[i, :],
        hovertext = df_Q.iloc[i, :]
    ), row = i // cols + 1, col = i % cols + 1)
    #print(i // 4 + 1, i % 4 + 1, df_Q.iloc[i, :])
#fig.update_layout(height = 300 + len(df_Q)//4 + 1, width=1000, title_text="Stacked subplots")
fig.update_layout(height = 100 * 10, 
                  width=200 * cols, 
                  title_text="Графики разложения отельных индивидов на компоненты",
                  showlegend = False
                 )
fig.show()